In [53]:
import pandas
import category_encoders as ce

df = pandas.read_csv("../DataSet/af2_dataset_training_labeled.csv")

In [54]:
# Convert feature true/false -> 1/0
binaryDataLst=['feat_A', 'feat_C', 'feat_D', 'feat_E', 'feat_F', 'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L', 'feat_M', 'feat_N', 'feat_P', 'feat_Q', 'feat_R',
               'feat_S', 'feat_T', 'feat_V', 'feat_W', 'feat_Y', 'feat_DSSP_H', 'feat_DSSP_B', 'feat_DSSP_E', 'feat_DSSP_G', 'feat_DSSP_I', 'feat_DSSP_T', 'feat_DSSP_S']

ram2 = df[binaryDataLst].astype('int')

In [55]:
# Convert entry to a set of range number
unit_range = df["entry"].unique()
Entry_list = df["entry"].replace(unit_range, list(range(1, len(unit_range) + 1)), inplace=False)

In [56]:
# construct min and max to normalize the already int column
minVals = {}
maxVals = {}
val_col = ['feat_PHI', 'feat_PSI', 'feat_TAU', 'feat_THETA', 'feat_BBSASA', 'feat_SCSASA', 'feat_pLDDT', 'feat_DSSP_6', 'feat_DSSP_7', 'feat_DSSP_8',
               'feat_DSSP_9', 'feat_DSSP_10', 'feat_DSSP_11', 'feat_DSSP_12', 'feat_DSSP_13', 'coord_X', 'coord_Y', 'coord_Z']
val_list = df[val_col].join(Entry_list)
val_col.append("entry")
normalized = pandas.DataFrame()
for i in val_col:
    minVals[i] = min(val_list[i])
    maxVals[i] = max(val_list[i])
    normalized[i] = (val_list[i] - minVals[i])/(maxVals[i] - minVals[i])

In [57]:
#join for output
output = ram2.join(normalized)

In [59]:
output.to_csv('../DataSet/normalized_input.csv')